<a href="https://colab.research.google.com/github/Adubi/BUS-DRIVER-ASSISTANCE-SYSTEM/blob/main/AUTONOMOUS_PUBLIC_BUS_DRIVER_ASSISTANCE_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load video from Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install gdown -U --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import gdown
!gdown 13Xvx-fZJb55vvT9tGaAa6ijE6fHgYIkl

Downloading...
From (uriginal): https://drive.google.com/uc?id=13Xvx-fZJb55vvT9tGaAa6ijE6fHgYIkl
From (redirected): https://drive.google.com/uc?id=13Xvx-fZJb55vvT9tGaAa6ijE6fHgYIkl&confirm=t&uuid=6b66b0c6-3f08-426b-a3fc-33d71f25ae64
To: /content/data.zip
100% 1.82G/1.82G [00:15<00:00, 116MB/s]


In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/13_6.txt           
  inflating: __MACOSX/data/._13_6.txt  
  inflating: data/35_5.txt           
  inflating: __MACOSX/data/._35_5.txt  
  inflating: data/70_5.txt           
  inflating: __MACOSX/data/._70_5.txt  
  inflating: data/10_6.jpg           
  inflating: __MACOSX/data/._10_6.jpg  
  inflating: data/36_5.jpg           
  inflating: __MACOSX/data/._36_5.jpg  
  inflating: data/73_5.jpg           
  inflating: __MACOSX/data/._73_5.jpg  
  inflating: data/54_4.txt           
  inflating: __MACOSX/data/._54_4.txt  
  inflating: data/0000001.jpg        
  inflating: __MACOSX/data/._0000001.jpg  
  inflating: data/1452_3.jpg         
  inflating: __MACOSX/data/._1452_3.jpg  
  inflating: data/105_4.jpg          
  inflating: __MACOSX/data/._105_4.jpg  
  inflating: data/0000015.jpg        
  inflating: __MACOSX/data/._0000015.jpg  
  inflating: data/140_4.jpg          
  inflating: __MACO

In [5]:
#get the video
!gdown 1JXsAxKjfTeBNvYRCV9ew1mAMccNklH4s

Downloading...
From: https://drive.google.com/uc?id=1JXsAxKjfTeBNvYRCV9ew1mAMccNklH4s
To: /content/VID-20230217-WA0000.mp4
100% 5.17M/5.17M [00:00<00:00, 261MB/s]


In [6]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

# Create folder in colab
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")

    # copying images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
train_test_split('/content/data/') # change to your dataset folder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
------ PROCESS STARTED -------
--- This folder has a total number of 569 images---


  0%|          | 0/456 [00:00<?, ?it/s]

------ Training data created with 80% split 456 images -------


  0%|          | 0/113 [00:00<?, ?it/s]

------ Testing data created with a total of 113 images ----------
------ TASK COMPLETED -------


In [7]:
!git clone https://github.com/kanyakornju/speed-prediction.git
%cd /content/speed-prediction/object-detection
!pip install -r requirements.txt

fatal: destination path 'speed-prediction' already exists and is not an empty directory.
/content/speed-prediction/object-detection
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!python setup.py

Checking setup...
YOLOv5 🚀 2023-6-25 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 31.0/78.2 GB disk)


In [9]:
!python train.py --epochs 20 --data dataset.yaml --cfg yolov5s.yaml

2023-06-25 07:31:58.776509: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 07:31:59.700786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interva

In [14]:
!python detect.py --save-txt --weights runs/train/exp2/weights/best.pt --conf 0.6 --source '/content/speed-prediction/object-detection/VID-20230217-WA0000.mp4'

detect: weights=['runs/train/exp2/weights/best.pt'], source=/content/speed-prediction/object-detection/VID-20230217-WA0000.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2023-6-25 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/738) /content/speed-prediction/object-detection/VID-20230217-WA0000.mp4: 384x640 5 cars, Done. (0.045s)
video 1/1 (2/738) /content/speed-prediction/object-detection/VID-20230217-WA0000.mp4: 384x640 6 cars, Done. (0.008s)
video 1/1 (3/738) /content/speed-prediction/object-detection/VID-20230217-WA0000.mp4: 